Create a Jupyter notebook to execute the following tasks, as part of the Big Data engineerig course:

1. Start a simple Spark session

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

import pandas as pd

2. Load the Walmart Stock CSV file, let Spark infer the data types

In [2]:
df = spark.read.options(infersSchema="true").csv("/content/walmart_stock.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



As we can see, Spark is not really good at infering the Schema. Lets manually coerce it:

In [3]:
df = spark.read.options(header="true").csv("/content/walmart_stock.csv")
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



Good! Now it works (more or less, see point 4)

3. Show the column names

In [4]:
df.schema.names

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

4. What does the Schema look like?

In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



However, as you can see in Point 5, not all columns are strings: some of them are dates, some are numbers... We will have to fix this later

5. Print out the first 5 rows

In [6]:
df.toPandas().head() #If I dont do toPandas(), I only print the first row, and the formatting is worse

,Date,Open,High,Low,Close,Volume,Adj Close
0,2012-01-03,59.970001,61.060001,59.869999,60.330002,12668800,52.619234999999996
1,2012-01-04,60.209998999999996,60.349998,59.470001,59.709998999999996,9593300,52.078475
2,2012-01-05,59.349998,59.619999,58.369999,59.419998,12768200,51.825539
3,2012-01-06,59.419998,59.450001,58.869999,59.0,8069400,51.45922
4,2012-01-09,59.029999,59.549999,58.919998,59.18,6679300,51.616215000000004


6. Use describe() to learn about the DataFrame

In [7]:
df.toPandas().describe() #If I do toPandas(), the shown info is cuter and more useful

,Date,Open,High,Low,Close,Volume,Adj Close
count,1258,1258,1258,1258,1258,1258,1258
unique,1258,957,956,938,943,1250,1184
top,2014-09-18,74.839996,75.190002,74.510002,73.510002,12653800,69.701339
freq,1,5,5,5,5,2,3


7. Format the numbers to show only 2 decimal places

In [8]:
#Moving it definitely toPandas() to make it easier to manage the df and set datatypes. 
df2 = df.toPandas().astype({'Date': "string", 'Open': float, 'High': float, 'Low': float, 'Close': float, 'Volume': float, 'Adj Close': float})
pd.options.display.float_format = '{:.2f}'.format
df2.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2012-01-03,59.97,61.06,59.87,60.33,12668800.00,52.62
1,2012-01-04,60.21,60.35,59.47,59.71,9593300.00,52.08
2,2012-01-05,59.35,59.62,58.37,59.42,12768200.00,51.83
3,2012-01-06,59.42,59.45,58.87,59.00,8069400.00,51.46
4,2012-01-09,59.03,59.55,58.92,59.18,6679300.00,51.62


8. Create a new DataFrame with a column called  'HV Ratio' that is the ratio of the High Price vs Volume of Stock traded for a day

In [9]:
newdf = df2; newdf['HV Ratio'] = newdf['High']/newdf['Volume']
newdf.head()

,Date,Open,High,Low,Close,Volume,Adj Close,HV Ratio
0,2012-01-03,59.97,61.06,59.87,60.33,12668800.00,52.62,0.00
1,2012-01-04,60.21,60.35,59.47,59.71,9593300.00,52.08,0.00
2,2012-01-05,59.35,59.62,58.37,59.42,12768200.00,51.83,0.00
3,2012-01-06,59.42,59.45,58.87,59.00,8069400.00,51.46,0.00
4,2012-01-09,59.03,59.55,58.92,59.18,6679300.00,51.62,0.00


You may think: It makes no sense! How can the HV Ratio be always 0? Dont worry: its just that we asked pandas to only show the first two decimals. If we ask it to show, for example, 10 decimal places:

In [10]:
pd.options.display.float_format = '{:.8f}'.format
newdf.head()

,Date,Open,High,Low,Close,Volume,Adj Close,HV Ratio
0,2012-01-03,59.97000100,61.06000100,59.86999900,60.33000200,12668800.00000000,52.61923500,0.00000482
1,2012-01-04,60.20999900,60.34999800,59.47000100,59.70999900,9593300.00000000,52.07847500,0.00000629
2,2012-01-05,59.34999800,59.61999900,58.36999900,59.41999800,12768200.00000000,51.82553900,0.00000467
3,2012-01-06,59.41999800,59.45000100,58.86999900,59.00000000,8069400.00000000,51.45922000,0.00000737
4,2012-01-09,59.02999900,59.54999900,58.91999800,59.18000000,6679300.00000000,51.61621500,0.00000892


9. What day had the Peak High in Price?

This is easy to do with sql:

In [11]:
df.createOrReplaceTempView("Wallmart")
spark.sql("SELECT Date FROM Wallmart where High = (SELECT MAX(High) from Wallmart);").show()

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+



10. What is the mean of the Close column

In [12]:
spark.sql("SELECT AVG(Close) from Wallmart;").show()

+--------------------------+
|avg(CAST(Close AS DOUBLE))|
+--------------------------+
|         72.38844998012726|
+--------------------------+



11. What is the max and min of the Volume column?

In [13]:
spark.sql("SELECT MIN(Volume), MAX(Volume) from Wallmart;").show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|   10010500|    9994400|
+-----------+-----------+



12. How many days was the Close lower than 60 dollars?

In [14]:
spark.sql("SELECT COUNT(Date) from Wallmart WHERE Close < 60;").show()

+-----------+
|count(Date)|
+-----------+
|         81|
+-----------+



13. What percentage of time was the High greater than 80 dollars?

In [15]:
spark.sql("SELECT COUNT(Date) from Wallmart WHERE High > 80;").show()

+-----------+
|count(Date)|
+-----------+
|        106|
+-----------+



14. What is the Pearson correlation between High and Volume?

This works better with Pandas:

In [16]:
newdf['High'].corr(newdf['Volume'])

-0.3384326061737164

15. What is the max High per year?

For this, we need to set the datatype of the date column as datetime, so that python can work with it; then, we can work with the code:

In [17]:
newdf['Date'] = pd.to_datetime(newdf['Date'], format='%Y-%m-%d')
newdf.groupby(newdf['Date'].dt.year)['High'].max()

Date
2012   77.59999800
2013   81.37000300
2014   88.08999600
2015   90.97000100
2016   75.19000200
Name: High, dtype: float64

16. What is the average Close for each calendar month?

In [18]:
newdf.groupby(newdf['Date'].dt.month)['Close'].mean()

Date
1    71.44801958
2    71.30680444
3    71.77794378
4    72.97361901
5    72.30971689
6    72.49537742
7    74.43971944
8    73.02981855
9    72.18411785
10   71.57854545
11   72.11108931
12   72.84792478
Name: Close, dtype: float64